<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/model%20/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/medical_noshow.csv')  

In [ ]:
df

,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4,AwaitingTime,Num_App_Missed,Gender_F,Gender_M
0,62,0,1,0,0,0,0,1,0,0,0,0,0.776481,0,1,0
1,56,0,0,0,0,0,0,1,0,0,0,0,0.672535,0,0,1
2,62,0,0,0,0,0,0,1,0,0,0,0,0.679907,0,1,0
3,8,0,0,0,0,0,0,1,0,0,0,0,0.728831,0,1,0
4,56,0,1,1,0,0,0,1,0,0,0,0,0.671794,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110514,56,0,0,0,0,1,0,1,0,0,0,0,34.614178,1,1,0
110515,51,0,0,0,0,1,0,1,0,0,0,0,34.689201,1,1,0
110516,21,0,0,0,0,1,0,1,0,0,0,0,40.330648,0,1,0
110517,38,0,0,0,0,1,0,1,0,0,0,0,40.368484,0,1,0


In [ ]:
x = df.loc[:, df.columns != 'No-show']
y = df[['No-show']]   

x_train, x_test, y_train,y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=42
)
     

In [ ]:
# 변수 설정
n_splits = 11   
random_state = 42
scaler = MinMaxScaler()

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler.fit(x_train)                 
x_train = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x_test = scaler.transform(x_test) 

In [ ]:
'''
#parameters_01
param = {
    'learning_rate': [0.1, 0.5, 1], # controls the learning rate
    'depth': [3, 4, 5], # controls the maximum depth of the tree
    'l2_leaf_reg': [2, 3, 4], # controls the L2 regularization term on weights
    'colsample_bylevel': [0.1, 0.2, 0.3], # specifies the fraction of columns to be randomly sampled for each level
    'n_estimators': [100, 200], # specifies the number of trees to be built
    'subsample': [0.1, 0.2, 0.3], # specifies the fraction of observations to be randomly sampled for each tree
    'border_count': [32, 64, 128],# specifies the number of splits for numerical features
    'bootstrap_type': ['Bernoulli', 'MVS']
} 

'''

#parameters_02
param = {
    'learning_rate': [0.1, 0.01, 0.001],
    'depth': [3, 5, 6, 9],
    'l2_leaf_reg': [1, 3, 5, 7]
} 


In [ ]:
cat = CatBoostClassifier()
model = GridSearchCV(cat, param,  cv = kfold, 
                   refit = True, verbose = 1, n_jobs = -1  )

#3. 훈련
import time
start_time = time.time()
model.fit(x_train, y_train)
end_time = time.time() - start_time

print('최적의 파라미터 : ', model.best_params_ )
print('최적의 매개변수 : ', model.best_estimator_)
print('best_score : ', model.best_score_)
print('model_score : ', model.score(x_test, y_test))
print('걸린 시간 : ', end_time, '초')

Fitting 11 folds for each of 48 candidates, totalling 528 fits


In [ ]:
# 2. 모델
model = CatBoostClassifier()

# 3. 훈련
model.fit(x_train, y_train)

#4. 평가, 예측
result = model.score(x_test,y_test)

score = cross_val_score( model, 
                        x_train, y_train,
                        cv = kfold )  #cv = cross validation_

y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv = kfold)

acc = accuracy_score(y_test, y_predict)

print('결과 acc : ', result)
print('cv pred acc : ', acc )

#결과 acc :  0.8005066497783407
# cv pred acc :  0.7988328960463222

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
6:	learn: 0.2867471	total: 44.5ms	remaining: 6.31s
7:	learn: 0.2660874	total: 56ms	remaining: 6.94s
8:	learn: 0.2500291	total: 61.3ms	remaining: 6.75s
9:	learn: 0.2378008	total: 68.2ms	remaining: 6.75s
10:	learn: 0.2266938	total: 73.6ms	remaining: 6.62s
11:	learn: 0.2184706	total: 78.8ms	remaining: 6.49s
12:	learn: 0.2114734	total: 84.4ms	remaining: 6.41s
13:	learn: 0.2059626	total: 89.6ms	remaining: 6.31s
14:	learn: 0.2013007	total: 94.7ms	remaining: 6.22s
15:	learn: 0.1970414	total: 99.8ms	remaining: 6.13s
16:	learn: 0.1936346	total: 105ms	remaining: 6.06s
17:	learn: 0.1906250	total: 110ms	remaining: 6s
18:	learn: 0.1881592	total: 115ms	remaining: 5.94s
19:	learn: 0.1859317	total: 120ms	remaining: 5.89s
20:	learn: 0.1841425	total: 125ms	remaining: 5.84s
21:	learn: 0.1824798	total: 130ms	remaining: 5.8s
22:	learn: 0.1809723	total: 135ms	remaining: 5.75s
23:	learn: 0.1797114	total: 141ms	remaining: 5.72s
24:	learn: 0.1786253	total: 145ms	remaining: 5